# Initialization

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, ArrayType, DateType
from pyspark.sql.functions import trim, col, lit

# Read from Bronze Table

In [0]:
df = spark.read.table("workspace.bronze.spotify_streaming_history_raw")
df.display()

# Silver Transformations

### Drop all NULL Columns

In [0]:
null_counts = df.select([F.count(c).alias(c) for c in df.columns]).collect()[0].asDict()
cols_to_drop = [k for k, v in null_counts.items() if v == 0]
df = df.drop(*cols_to_drop)

print(f"Colunas removidas: {cols_to_drop}")

### Remove rows where Track ID is null
- These rows are relative to episode podcasts

In [0]:
df = df.dropna(subset=["spotify_track_uri"])

### Trimming

In [0]:
for field in df.schema.fields:
  if isinstance(field.dataType, StringType):
    df = df.withColumn(field.name, trim(col(field.name)))
  elif isinstance(field.dataType, ArrayType):
    df = df.withColumn(field.name, F.transform(col(field.name), lambda x: trim(x)))

### Replace Track URI for Track ID

In [0]:
df = df.withColumn("track_id", F.split(F.col("spotify_track_uri"), ":")[2]).drop("spotify_track_uri")

### Drop Unnecessary Columns
- Columns relative to **Track**, **Album** and **Artist**, because of the star schema that it will be used to get that information
- Columns relative to podcasts

In [0]:
cols_to_drop = [
    "master_metadata_album_artist_name",
    "master_metadata_album_album_name",
    "master_metadata_track_name",
    "episode_name",
    "episode_show_name",
    "spotify_episode_uri"
]

df = df.drop(*cols_to_drop)

### Convert Date Columns to Timestamp

In [0]:
df = df.withColumn("ts", F.to_timestamp(col("ts")))

### Handle Offline Column Values

In [0]:
df = df.withColumn("offline_timestamp", 
    F.when(
        (F.col("offline") == True) & (F.col("offline_timestamp") > 0), 
        F.col("offline_timestamp")
    ).otherwise(None)
)

# Se o valor tiver 13 dígitos, divide-se por 1000. Se tiver 10, usa-se o valor
df = df.withColumn("offline_at", 
    F.when(F.length(F.col("offline_timestamp").cast("string")) >= 13, 
           F.from_unixtime(F.col("offline_timestamp") / 1000))
     .otherwise(F.from_unixtime(F.col("offline_timestamp"))).cast("timestamp")
)

### Normalizing

Country Names

In [0]:
df = df.withColumn(
    "country",
    F.when(F.upper(F.col("conn_country")) == "PT", "Portugal")
     .when(F.upper(F.col("conn_country")) == "BR", "Brazil")
     .when(F.upper(F.col("conn_country")) == "US", "United States")
     .when(F.upper(F.col("conn_country")) == "JP", "Japan")
     .when(F.upper(F.col("conn_country")) == "NL", "Netherlands")
     .when(F.upper(F.col("conn_country")) == "DE", "Germany")
     .when(F.upper(F.col("conn_country")) == "AT", "Austria")
     .when(F.upper(F.col("conn_country")).isin("UK", "GB"), "United Kingdom")
     .when(F.upper(F.col("conn_country")) == "ES", "Spain")
     .when(F.upper(F.col("conn_country")) == "FR", "France")
     .when(F.upper(F.col("conn_country")) == "IT", "Italy")
     .when(F.upper(F.col("conn_country")) == "ZZ", "n/a")
     .otherwise(col("conn_country"))
)

Start and End Reason

In [0]:
df = df.withColumn(
    "reason_start",
    F.when(col("reason_start") == "appload", "app_load")
     .when(col("reason_start") == "backbtn", "back_button")
     .when(col("reason_start") == "fwdbtn", "forward_button")
     .when(col("reason_start") == "trackerror", "track_error")
     .when(col("reason_start") == "trackdone", "track_done")
     .when(col("reason_start") == "clickrow", "click_row")
     .when(col("reason_start") == "remote", "remote")
     .when(col("reason_start") == "unknown", "unknown")
     .when(col("reason_start") == "playbtn", "play_button")
     .when(col("reason_start") == "switched-to-audio", "switched_to_audio")
     .otherwise(col("reason_start"))
)

df = df.withColumn(
    "reason_end",
    F.when(col("reason_end") == "appload", "app_load")
     .when(col("reason_end") == "backbtn", "back_button")
     .when(col("reason_end") == "endplay", "end_play")
     .when(col("reason_end") == "fwdbtn", "forward_button")
     .when(col("reason_end") == "logout", "logout")
     .when(col("reason_end") == "remote", "remote")
     .when(col("reason_end") == "trackdone", "track_done")
     .when(col("reason_end") == "trackerror", "track_error")
     .when(col("reason_end") == "unexpected-exit", "unexpected_exit")
     .when(col("reason_end") == "unexpected-exit-while-paused", "unexpected_exit_while_paused")
     .when(col("reason_end") == "unknown", "unknown")
     .otherwise(col("reason_end"))
)

### Rename Columns

In [0]:
RENAME_MAP = {
    "conn_country": "country_code",
    "ip_addr": "ip_address",
    "ts": "played_at",
    "reason_start": "start_reason",
    "reason_end": "end_reason"
}

for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

### Handle NULL Values

In [0]:
df = df.withColumns({
    "country_code": F.coalesce(col("country_code"), lit("n/a")),
    "incognito_mode": F.coalesce(col("incognito_mode"), lit(False)),
    "ip_address": F.coalesce(col("ip_address"), lit("n/a")),
    "ms_played": F.coalesce(col("ms_played"), lit(0)),
    "offline": F.coalesce(col("offline"), lit(False)),
    "offline_timestamp": F.coalesce(col("offline_timestamp"), lit(0)),
    "platform": F.coalesce(col("platform"), lit("n/a")),
    "end_reason": F.coalesce(col("end_reason"), lit("n/a")),
    "start_reason": F.coalesce(col("start_reason"), lit("n/a")),
    "shuffle": F.coalesce(col("shuffle"), lit(True)),
    "skipped": F.coalesce(col("skipped"), lit(False)),
    "played_at": F.coalesce(F.col("played_at"), F.to_timestamp(F.lit("1900-01-01 00:00:00"))),
    "processed_at": F.coalesce(F.col("processed_at"), F.to_timestamp(F.lit("1900-01-01 00:00:00"))),
    "track_id": F.coalesce(col("track_id"), lit("n/a")),
    "offline_at": F.coalesce(F.col("processed_at"), F.to_timestamp(F.lit("1900-01-01 00:00:00"))),
    "country": F.coalesce(col("country"), lit("n/a"))
})

### Column Order

In [0]:
column_order = [
    "played_at",
    "track_id",
    "ms_played",
    "start_reason",
    "end_reason",
    "shuffle",
    "skipped",
    "incognito_mode",
    "offline",
    "offline_at",
    "offline_timestamp",
    "platform",
    "country",
    "ip_address",
    "processed_at"
]

df = df.select(*column_order)

## Check Dataframe

In [0]:
df.display()

# Save Silver Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.spotify_streaming_history")